In [4]:
from openai import OpenAI
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

client = OpenAI()


In [6]:
response = client.responses.create(
    model="gpt-4.1-mini",
    input=[{
        "role": "user",
        "content": [
            {"type": "input_text", "text": "여기 나오는 강아지는 어떤 품종이야?"},
            {
                "type": "input_image",
                "image_url": "https://cdn.pixabay.com/photo/2025/09/12/16/42/dog-9830812_1280.jpg",
            },
        ],
    }],
)

print(response.output_text)

이 강아지들은 시바 이누(Shiba Inu) 품종입니다. 시바 이누는 일본 원산의 작고 귀여운 견종으로, 쫑긋한 귀와 말린 꼬리, 그리고 풍성한 털이 특징입니다.


In [7]:
import base64

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


# Path to your image
image_path = "dog-9830812_1280.jpg"

# Getting the Base64 string
base64_image = encode_image(image_path)

response = client.responses.create(
    model="gpt-4.1-mini",
    input=[{
        "role": "user",
        "content": [
            {"type": "input_text", "text": "이건 어떤 종의 강아지 사진이야?"},
            {
                "type": "input_image",
                "image_url": f"data:image/jpeg;base64,{base64_image}",
                "detail": "high"
            },
        ],
    }],
)

print(response.output_text)

이 강아지들은 시바 이누(Shiba Inu) 종입니다. 시바 이누는 일본 원산의 중소형 견종으로, 붉은색의 털과 쫑긋한 귀, 그리고 꼬리가 둥글게 말려 있는 특징이 있습니다. 사진 속 강아지들도 그런 특징들을 잘 보여주고 있네요.


In [8]:
# Function to create a file with the Files API
def create_file(file_path):
  with open(file_path, "rb") as file_content:
    result = client.files.create(
        file=file_content,
        purpose="vision",
    )
    return result.id

# Getting the file ID
file_id = create_file("dog-9830812_1280.jpg")

response = client.responses.create(
    model="gpt-4.1-mini",
    input=[{
        "role": "user",
        "content": [
            {"type": "input_text", "text": "이건 어떤 종의 강아지 사진이야?"},
            {
                "type": "input_image",
                "file_id": file_id,
            },
        ],
    }],
)

print(response.output_text)

이 사진 속 강아지들은 시바 이누(Shiba Inu) 종입니다. 시바 이누는 일본 원산의 소형 견종으로, 귀여운 외모와 활발한 성격으로 인기가 많아요.


In [9]:

response = client.responses.create(
    model="gpt-4.1-mini",
    input="한국에서 유행중인 두쫀쿠 그림 그려줘.",
    tools=[{"type": "image_generation"}],
)

#  Save the image to a file
image_data = [
    output.result
    for output in response.output
    if output.type == "image_generation_call"
]

if image_data:
    image_base64 = image_data[0]
    with open("desert.png", "wb") as f:
        f.write(base64.b64decode(image_base64))

In [10]:
import base64

img = client.images.generate(
    model="gpt-image-1.5",
    prompt="죽순 먹는 팬더 그림",
    n=1,
    size="1024x1024"
)

image_bytes = base64.b64decode(img.data[0].b64_json)
with open("output.png", "wb") as f:
    f.write(image_bytes)


In [11]:
audio_file = open("/content/drive/MyDrive/ElevenLabs_Text_to_Speech_audio.mp3", "rb")
transcript = client.audio.transcriptions.create(
  model="gpt-4o-transcribe",
  file=audio_file
)

print(transcript.text)

이건 예제 음성 파일입니다. 이 내용을 텍스트로 변환하는 작업을 STT, Speech to Text라고 하고요.


In [14]:
from pathlib import Path
from IPython.display import Audio

text = "오늘은 OpenAI의 텍스트 음성 변환 기능을 테스트해 보는 중입니다."

output_path = Path("tts_sample.mp3")

with client.audio.speech.with_streaming_response.create(
    model="gpt-4o-mini-tts",
    voice="marin",               # 마음에 드는 기본 보이스 선택 (alloy, coral 등)
    input=text,
    instructions="급하고 흥분한 사람의 한국어 발음으로 읽어줘.",
    speed = 2.0
) as response:
    response.stream_to_file(output_path)

print("생성된 파일:", output_path)

Audio(filename=str(output_path))

생성된 파일: tts_sample.mp3


In [15]:
from pathlib import Path
from IPython.display import Audio, display

input_audio_path = "/content/drive/MyDrive/TODO_speech.mp3"

# Speech -> Text (audio.transcriptions)
with open(input_audio_path, "rb") as f:
    transcription = client.audio.transcriptions.create(
        model="gpt-4o-transcribe",  # 또는 gpt-4o-mini-transcribe 등 :contentReference[oaicite:0]{index=0}
        file=f,
        # language="ko",  # 한국어 음성이면 선택적으로 명시해도 됨
    )

original_text = transcription.text
print("=== 1) 원본 음성에서 얻은 텍스트 ===")
print(original_text)

# Responses API로 텍스트 처리 (요약/정리/답변 등)
system_prompt = """\
너는 친절한 한국인 비서다.
사용자의 발화를 이해해서,
1) 한 줄 요약
2) 할 일(To-do) 목록
을 한국어로 정리해줘.
"""

prompt = f"{system_prompt}\n\n[사용자 발화]\n{original_text}"

resp = client.responses.create(
    model="gpt-4.1-mini",
    input=prompt,
)

final_text = resp.output_text
print("\n=== 2) LLM 처리 결과(최종 텍스트) ===")
print(final_text)

# Text -> Speech (audio.speech, TTS)
output_audio_path = Path("speech_reply.mp3")

with client.audio.speech.with_streaming_response.create(
    model="gpt-4o-mini-tts",        # 권장 TTS 모델
    voice="marin",                  # 또는 alloy / cedar / coral 등
    input=final_text,
    instructions="차분하고 자연스러운 한국어로 읽어줘.",
) as response:
    response.stream_to_file(output_audio_path)

print("\n=== 3) 생성된 음성 파일 ===")
print(output_audio_path)


display(Audio(filename=str(output_audio_path)))




=== 1) 원본 음성에서 얻은 텍스트 ===
오늘 공항에 가서 부모님 마중한 다음 모셔다 드리고 대파랑 삼겹살 사서 집에 와야 해. 아, 재활용품 분리수거하는 날이네.

=== 2) LLM 처리 결과(최종 텍스트) ===
1) 한 줄 요약  
오늘 공항에 부모님 마중하고, 대파와 삼겹살 사서 집에 오며 재활용품 분리수거까지 해야 한다.

2) 할 일(To-do) 목록  
- 공항으로 부모님 마중 가기  
- 부모님 모시고 집으로 돌아오기  
- 대파와 삼겹살 구매하기  
- 재활용품 분리수거 하기

=== 3) 생성된 음성 파일 ===
speech_reply.mp3
